# Azure DocumentDB Customer 360 App Simulation

This notebook demonstrates how to build a modern, intelligent customer data platform using Azure DocumentDB.

## Objectives
- Provision DocumentDB database and collections required.
- Generate hundreds of customers with their bank and credit card transactions data.
- Use various Aggregation pipelines 
- Incorporate Graph lookups
- Power BI style dashboards for real-time insights
- Vector Embeddings + Retrieval-Augmented Generation (RAG) with Azure OpenAI

## Prerequisites
- Azure DocumentDB cluster setup.
- Python 3.9+ environment with Jupyter Notebook.
- Azure DocumentDB connection string, Open AI endpoint and key.
- Ensure proper permissions to write input files under the data folder, and create a .env file for the credentials required.
- Please refer to this link to create python environments in VS Code, [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments)

In [ ]:
from faker import Faker
import random
import pandas as pd
import os
from datetime import datetime
from pymongo import MongoClient
import pandas as pd
from dotenv import dotenv_values
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import quote_plus
import networkx as nx

env_name = "./config/config.env" # using config.env template, change to your own .env file name
config = dotenv_values(env_name)

connection_string = config['DOCUMENTDB_CONN_STRING']
cust_file_path = "./data/customers.csv"
bank_trans_file_path = "./data/customer_bank_transactions.csv"
cc_trans_file_path = "./data/customer_credit_card_transactions.csv"
db_name = "customer360"

## Generate Sample Customer 360 Data with Bank and Credit Card Transactions

In [ ]:
def format_elapsed_time(elapsed_seconds):
    hours, remainder = divmod(elapsed_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02}:{int(minutes):02}:{seconds:05.2f}"

start_time = datetime.now()

fake = Faker()
Faker.seed(0)
random.seed(0)

# Generate synthetic customer profiles
def generate_customers(num_customers):
    customers = []
    for i in range(num_customers):
        customer_id = f"C{1000 + i}"
        customers.append({
            "customer_id": customer_id,
            "name": fake.name(),
            "email": fake.email(),
            "phone": fake.phone_number(),
            "address": fake.address().replace("\n", ", "),
            "dob": fake.date_of_birth(minimum_age=18, maximum_age=80).strftime("%Y-%m-%d"),
            "account_open_date": fake.date_between(start_date='-10y', end_date='today').strftime("%Y-%m-%d")
        })
    return customers

# Generate synthetic bank transactions
def generate_bank_transactions(customers, num_transactions):
    transactions = []
    transaction_types = ["deposit", "withdrawal", "transfer"]
    for i in range(num_transactions):
        customer = random.choice(customers)
        transactions.append({
            "transaction_id": f"B{10000 + i}",
            "customer_id": customer["customer_id"],
            "date": fake.date_between(start_date='-2y', end_date='today').strftime("%Y-%m-%d"),
            "type": random.choice(transaction_types),
            "amount": round(random.uniform(50, 5000), 2),
            "description": fake.sentence()
        })
    return transactions

# Generate synthetic credit card transactions
def generate_credit_card_transactions(customers, num_transactions):
    transactions = []
    categories = ["groceries", "travel", "electronics", "restaurants", "clothing", "utilities"]
    for i in range(num_transactions):
        customer = random.choice(customers)
        transactions.append({
            "transaction_id": f"CC{20000 + i}",
            "customer_id": customer["customer_id"],
            "date": fake.date_between(start_date='-2y', end_date='today').strftime("%Y-%m-%d"),
            "merchant": fake.company(),
            "category": random.choice(categories),
            "amount": round(random.uniform(10, 2000), 2),
            "description": fake.sentence()
        })
    return transactions

# Generate and save datasets
num_customers = 100
num_bank_transactions = 500
num_credit_card_transactions = 500

customers = generate_customers(num_customers)
bank_transactions = generate_bank_transactions(customers, num_bank_transactions)
credit_card_transactions = generate_credit_card_transactions(customers, num_credit_card_transactions)

# Ensure the directory exists if not create it
os.makedirs(os.path.dirname(cust_file_path), exist_ok=True)

os.makedirs(os.path.dirname(bank_trans_file_path), exist_ok=True)

os.makedirs(os.path.dirname(cc_trans_file_path), exist_ok=True)

pd.DataFrame(customers).to_csv(cust_file_path, index=False)
pd.DataFrame(bank_transactions).to_csv(bank_trans_file_path, index=False)
pd.DataFrame(credit_card_transactions).to_csv(cc_trans_file_path, index=False)

print("Synthetic datasets saved as CSV files.")

end_time = datetime.now()
print(f"Generated customer transactions and saved to {cust_file_path, bank_trans_file_path, cc_trans_file_path}")

elapsed_time = end_time - start_time
elapsed_time_str = format_elapsed_time(elapsed_time.total_seconds())
print(f"Total time taken to generate the input file HH:MM:SS: {elapsed_time_str}")

## Ingest Data into Azure DocumentDB

In [ ]:
# Replace with your Azure DocumentDB connection string
client = MongoClient(connection_string)
client.drop_database(db_name) #drop if DB exists and recreate
db = client[db_name]

# Load CSVs
customers_df = pd.read_csv(cust_file_path)
bank_tx_df = pd.read_csv(bank_trans_file_path)
cc_tx_df = pd.read_csv(cc_trans_file_path)

start_time = datetime.now()
# Insert into DocumentDB, provide the respective collection names
db.customers.insert_many(customers_df.to_dict(orient="records"))
db.customer_bank_trans.insert_many(bank_tx_df.to_dict(orient="records"))
db.customer_card_trans.insert_many(cc_tx_df.to_dict(orient="records"))

end_time = datetime.now()
elapsed_time = end_time - start_time
elapsed_time_str = format_elapsed_time(elapsed_time.total_seconds())
print(f"Total time taken to insert the input files to Mongo vCore collections, HH:MM:SS: {elapsed_time_str}")

## Build various aggregation query pipelines
### # 1. Customers with accounts older than 5 years

In [ ]:
from pprint import pprint
from datetime import datetime, UTC

now = datetime.now(UTC)

# 1. Customers with accounts older than 5 years
print("\n1. Customers with accounts older than 5 years:")
result1 = db.customers.aggregate([
    {
        "$addFields": {
            "account_open_date_dt": {
                "$cond": {
                    "if": { "$eq": [{ "$type": "$account_open_date" }, "string"] },
                    "then": { "$dateFromString": { "dateString": "$account_open_date" } },
                    "else": "$account_open_date"
                }
            }
        }
    },
    {
        "$addFields": {
            "account_age": {
                "$divide": [
                    { "$subtract": [now, "$account_open_date_dt"] },
                    1000 * 60 * 60 * 24 * 365  # Convert milliseconds to years
                ]
            }
        }
    },
    {
        "$match": {
            "account_age": { "$gt": 5 }
        }
    },
    {
        "$sort": { "account_age": -1 }
    },
    {
        "$limit": 5
    }
])
pprint(list(result1))

### 2. Credit card transactions over $300 in groceries

In [ ]:
# 2. Credit card transactions over $300 in groceries
print("\n2. Credit card transactions over $300 in groceries:")
result2 = db.customer_card_trans.aggregate([
    {
        "$match": {
            "amount": { "$gt": 300 },
            "category": "groceries"
        }
    }, { "$limit": 5 }

])
pprint(list(result2))

### 3. Bank withdrawals over $200

In [ ]:
# 3. Bank withdrawals over $200
print("\n3. Bank withdrawals over $200:")
result3 = db.customer_bank_trans.aggregate([
    {
        "$match": {
            "type": "withdrawal",
            "amount": { "$gt": 200 }
        }
    }, { "$limit": 5 }
])
pprint(list(result3))

### 4. Group transactions by customer and count

In [ ]:
# 4. Group transactions by customer and count
print("\n4. Group transactions by customer and count:")
result4 = db.customer_bank_trans.aggregate([
    {
        "$group": {
            "_id": "$customer_id",
            "total_transactions": { "$sum": 1 },
            "total_amount": { "$sum": "$amount" }
        }
    },
    {
        "$sort": { "total_amount": -1 }
    }, { "$limit": 5 }
])
pprint(list(result4))

### 5. Monthly spending summary for credit cards

In [ ]:
# 5. Monthly spending summary for credit cards
print("\n5. Monthly spending summary for credit cards:")
result5 = db.customer_card_trans.aggregate([
    {
        "$addFields": {
            "month": { "$month": { "$dateFromString": { "dateString": "$date" } } },
            "year": { "$year": { "$dateFromString": { "dateString": "$date" } } }
        }
    },
    {
        "$group": {
            "_id": { "year": "$year", "month": "$month" },
            "total_spent": { "$sum": "$amount" },
            "transaction_count": { "$sum": 1 }
        }
    },
    {
        "$sort": { "_id.year": -1, "_id.month": -1 }
    }
    , 
    { "$limit": 5 }
])
pprint(list(result5))

### 6. Join customers with their transactions using $lookup

In [ ]:
# 6. Join customers with their transactions using $lookup
print("\n6. Join customers with their transactions using $lookup:")
result6 = db.customers.aggregate([
    {
        "$lookup": {
            "from": "bank_transactions",
            "localField": "customer_id",
            "foreignField": "customer_id",
            "as": "bank_txns"
        }
    },
    {
        "$lookup": {
            "from": "credit_card_transactions",
            "localField": "customer_id",
            "foreignField": "customer_id",
            "as": "credit_txns"
        }
    }
    , { "$limit": 5 }
])
pprint(list(result6))

## Use $graphLookup feature traversal for Credit Card and Bank Transactions

In [ ]:
# Define the sample customer_id
sample_customer_id = "C1001"  # Replace with a valid customer_id from your dataset

# Graph lookup pipeline to retrieve both credit card and bank transactions
pipeline = [
    {
        "$match": {"customer_id": sample_customer_id}
    },
    {
        "$graphLookup": {
            "from": "customer_card_trans",
            "startWith": "$customer_id",
            "connectFromField": "customer_id",
            "connectToField": "customer_id",
            "as": "related_credit_card_transactions"
        }
    },
    {
        "$graphLookup": {
            "from": "customer_bank_trans",
            "startWith": "$customer_id",
            "connectFromField": "customer_id",
            "connectToField": "customer_id",
            "as": "related_bank_transactions"
        }
    },
    {
        "$project": {
            "customer_id": 1,
            "name": 1,
            "related_credit_card_transactions": 1,
            "related_bank_transactions": 1
        }
    }
]

# Execute the aggregation
graph_result = list(db.customers.aggregate(pipeline))

# Print results
if graph_result:
    print(f"🔍 Related credit card transactions for customer {sample_customer_id}:")
    for txn in graph_result[0]["related_credit_card_transactions"]:
        print(txn)

    print(f"\n🏦 Related bank transactions for customer {sample_customer_id}:")
    for txn in graph_result[0]["related_bank_transactions"]:
        print(txn)
else:
    print(f"❌ No customer found with ID {sample_customer_id}")

## Graph Visualization using Python libraries, similar approach can be used on other Graph Visualization tools

In [ ]:
# Create graph
G = nx.Graph()

# print(graph_result)
customer_data = graph_result[0]

# Add customer node
customer_id = customer_data["customer_id"]
customer_name = customer_data["name"]
G.add_node(customer_id, label=customer_name, type="customer")

# Add credit card transaction nodes and edges
for tx in customer_data["related_credit_card_transactions"]:
    tx_id = tx["transaction_id"]
    tx_label = f"Card: {tx['category']} ${tx['amount']}"
    G.add_node(tx_id, label=tx_label, type="credit_card_tx")
    G.add_edge(customer_id, tx_id)

# Add bank transaction nodes and edges
for tx in customer_data["related_bank_transactions"]:
    tx_id = tx["transaction_id"]
    tx_label = f"Bank: {tx['type']} ${tx['amount']}"
    G.add_node(tx_id, label=tx_label, type="bank_tx")
    G.add_edge(customer_id, tx_id)

# Draw graph
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)

# Assign colors based on node type
node_colors = []
for node, data in G.nodes(data=True):
    if data["type"] == "customer":
        node_colors.append("skyblue")
    elif data["type"] == "bank_tx":
        node_colors.append("lightgreen")
    else:
        node_colors.append("salmon")

# Draw nodes and edges
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=600)
nx.draw_networkx_edges(G, pos, alpha=0.5)

# Draw labels
labels = {node: data["label"] for node, data in G.nodes(data=True)}
nx.draw_networkx_labels(G, pos, labels, font_size=9)

plt.title("Customer Transaction Graph")
plt.axis("off")
plt.tight_layout()
plt.show()

## Graph Visualization of Customer Transactions
### Customers → Transactions → Merchants
### Customers → Transactions → Categories

In [ ]:

# Create graph
G = nx.Graph()

# print(graph_result)
customer_data = graph_result[0]

# Add customer node
customer_id = customer_data["customer_id"]
G.add_node(customer_id, label=customer_data["name"], type="customer")

# Add credit card transactions and merchant/category/offer nodes
for tx in customer_data["related_credit_card_transactions"]:
    tx_id = tx["transaction_id"]
    tx_label = f"Card: {tx['category']} ${tx['amount']}"
    merchant = tx["merchant"]
    category = tx["category"]
    # offer = customer_data["offers"].get(category, "No offer")

    # Transaction node
    G.add_node(tx_id, label=tx_label, type="credit_card_tx")
    G.add_edge(customer_id, tx_id)

    # Merchant node
    G.add_node(merchant, label=merchant, type="merchant")
    G.add_edge(tx_id, merchant)

    # Category node
    G.add_node(category, label=category, type="category")
    G.add_edge(tx_id, category)

    # Offer node
    # G.add_node(offer, label=offer, type="offer")
    # G.add_edge(category, offer)

# Add bank transactions
for tx in customer_data["related_bank_transactions"]:
    tx_id = tx["transaction_id"]
    tx_label = f"Bank: {tx['type']} ${tx['amount']}"
    G.add_node(tx_id, label=tx_label, type="bank_tx")
    G.add_edge(customer_id, tx_id)

# Draw graph
plt.figure(figsize=(14, 12))
pos = nx.spring_layout(G, seed=42)

# Assign colors based on node type
node_colors = []
for node, data in G.nodes(data=True):
    if data["type"] == "customer":
        node_colors.append("skyblue")
    elif data["type"] == "bank_tx":
        node_colors.append("lightgreen")
    elif data["type"] == "credit_card_tx":
        node_colors.append("salmon")
    elif data["type"] == "merchant":
        node_colors.append("orange")
    elif data["type"] == "category":
        node_colors.append("violet")
    elif data["type"] == "offer":
        node_colors.append("gold")
    else:
        node_colors.append("gray")

# Draw nodes and edges
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=600)
nx.draw_networkx_edges(G, pos, alpha=0.5)

# Draw labels
labels = {node: data["label"] for node, data in G.nodes(data=True)}
nx.draw_networkx_labels(G, pos, labels, font_size=8)

plt.title("Extended Customer Transaction Graph")
plt.axis("off")
plt.tight_layout()
plt.show()

## Customer Segmentation - Power BI-style dashboards using matplotlib and seaborn.
### DocumentDB can be integrated to Power BI for generating such dashboards.

In [ ]:
# Load data from MongoDB collections
customers_df = pd.DataFrame(list(db.customers.find()))
bank_tx_df = pd.DataFrame(list(db.customer_bank_trans.find()))
cc_tx_df = pd.DataFrame(list(db.customer_card_trans.find()))

# Drop MongoDB internal _id field
for df in [customers_df, bank_tx_df, cc_tx_df]:
    df.drop(columns=["_id"], inplace=True, errors="ignore")

# Combine bank and credit card transactions
bank_tx_df["source"] = "bank"
cc_tx_df["source"] = "credit_card"
combined_tx_df = pd.concat([bank_tx_df, cc_tx_df], ignore_index=True)

# Calculate total spending per customer
spending_summary = combined_tx_df.groupby("customer_id")["amount"].sum().reset_index()
spending_summary.rename(columns={"amount": "total_spending"}, inplace=True)

# Merge with customer profiles
customer_summary = pd.merge(customers_df, spending_summary, on="customer_id", how="left")
customer_summary["total_spending"].fillna(0, inplace=True)

# Segment customers based on total spending
def segment_customer(spending):
    if spending < 500:
        return "Low"
    elif spending < 2000:
        return "Medium"
    elif spending < 5000:
        return "High"
    else:
        return "Premium"

customer_summary["segment"] = customer_summary["total_spending"].apply(segment_customer)

# Plot 1: Spending distribution by segment
plt.figure(figsize=(8, 6))
sns.boxplot(x="segment", y="total_spending", data=customer_summary, order=["Low", "Medium", "High", "Premium"])
plt.title("Customer Spending Distribution by Segment")
plt.xlabel("Segment")
plt.ylabel("Total Spending")
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot 2: Count of customers by segment
plt.figure(figsize=(6, 5))
sns.countplot(x="segment", data=customer_summary, order=["Low", "Medium", "High", "Premium"], palette="Set2")
plt.title("Customer Count by Segment")
plt.xlabel("Segment")
plt.ylabel("Number of Customers")
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot 3: Total spending by transaction source
plt.figure(figsize=(6, 5))
sns.barplot(x="source", y="amount", data=combined_tx_df, estimator=sum, ci=None, palette="Set1")
plt.title("Total Spending by Transaction Source")
plt.xlabel("Source")
plt.ylabel("Total Amount")
plt.grid(True)
plt.tight_layout()
plt.show()

## Generate Vector Embeddings & Implement Retrieval-Augmented Generation (RAG)

In [ ]:
import openai
import numpy as np
from pymongo import MongoClient

# Step 1: Configure Azure OpenAI
openai.api_type = "azure"
openai.api_key = config['OPENAI_API_KEY']
openai.api_base = config['OPENAI_API_ENDPOINT']
openai.azure_endpoint = config['OPENAI_API_ENDPOINT']
openai.api_version = "2023-05-15"
embedding_model = config['OPENAI_EMBEDDINGS_DEPLOYMENT']

# Step 2: Connect to MongoDB
client = MongoClient(connection_string)
db = client[db_name]

# Step 3: Read data from collections
customer_profiles = list(db["customers"].find().limit(10))
db["customer_embeddings"].drop()

# Step 4: Embed and store documents
def embed_and_store(text, metadata):
    response = openai.embeddings.create(
        input=[text],
        model=embedding_model
    )
    embedding = response.data[0].embedding
    doc = {
        "text": text,
        "embedding": embedding,
        "metadata": metadata
    }
    db["customer_embeddings"].insert_one(doc)



# Step 5: Process each customer and their transactions
for customer in customer_profiles:
    customer_id = customer.get("customer_id")

    # Embed customer profile
    profile_text = (
        f"Customer ID: {customer.get('customer_id')}; "
        f"Name: {customer.get('name')}; "
        f"Email: {customer.get('email')}; "
        f"Phone: {customer.get('phone')}; "
        f"Address: {customer.get('address')}; "
        f"Date of Birth: {customer.get('dob')}; "
        f"Account Open Date: {customer.get('account_open_date')}"
    )
    embed_and_store(profile_text, {"type": "customer_profile", "customer_id": customer_id})

    # Embed bank transactions
    bank_transactions = list(db["customer_bank_trans"].find({"customer_id": customer_id}))
    for txn in bank_transactions:
        txn_text = (
            f"Bank Transaction ID: {txn.get('transaction_id')}; "
            f"Customer ID: {txn.get('customer_id')}; "
            f"Date: {txn.get('date')}; "
            f"Type: {txn.get('type')}; "
            f"Amount: {txn.get('amount')}; "
            f"Description: {txn.get('description')}"
        )
        embed_and_store(txn_text, {"type": "bank_transaction", "customer_id": customer_id})

    # Embed credit card transactions
    credit_card_transactions = list(db["customer_card_trans"].find({"customer_id": customer_id}))
    for cc_txn in credit_card_transactions:
        cc_text = (
            f"Credit Card Transaction ID: {cc_txn.get('transaction_id')}; "
            f"Customer ID: {cc_txn.get('customer_id')}; "
            f"Date: {cc_txn.get('date')}; "
            f"Merchant: {cc_txn.get('merchant')}; "
            f"Category: {cc_txn.get('category')}; "
            f"Amount: {cc_txn.get('amount')}; "
            f"Description: {cc_txn.get('description')}"
        )
        embed_and_store(cc_text, {"type": "credit_card_transaction", "customer_id": customer_id})


# Step 5: Semantic search

def semantic_search(query, top_k=3):
    query_embedding = openai.embeddings.create(
        input=[query],
        model=embedding_model
    ).data[0].embedding

    results = []
    for doc in db["customer_embeddings"].find():
        doc_embedding = np.array(doc["embedding"])
        similarity = np.dot(query_embedding, doc_embedding) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding)
        )
        results.append((similarity, doc["text"]))
    results.sort(reverse=True, key=lambda x: x[0])
    return results[:top_k]

# Step 6: Sample RAG queries
sample_queries = [
    "Which customers have frequent withdrawals?",
    "Find customers with high-value transactions across both bank and credit card",
    "Which customers have unusual transaction description?",
    "Find credit card transactions in the groceries category.",
    "Who are the customers with consistent spending patterns?"
]

# Step 7: Run queries
for query in sample_queries:
    print(f"\nQuery: {query}")
    results = semantic_search(query)
    for score, text in results:
        print(f"Score: {score:.4f} | Text: {text}")
